# GSEA with gseapy

In [1]:
#Boilerplate Import 
import sys
sys.path.append('..')
import os
from itertools import chain
from collections import defaultdict
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from scripts import utils
from scripts.analysis import build_loci

In [2]:
BASE = '../data'

unenr_grouped_loci = utils.load(os.path.join(BASE,"unenriched_grouped_loci.pkl.gz"))
enr_grouped_loci = utils.load(os.path.join(BASE,"enriched_grouped_loci_filt1.pkl.gz"))
grouped_loci = utils.load(os.path.join(BASE,"grouped_loci_filt1.pkl.gz"))

In [3]:
import gseapy as gp
import goatools

obodag = goatools.obo_parser.GODag('go-basic.obo')

load obo file go-basic.obo
go-basic.obo: fmt(1.2) rel(2016-07-16) 46,953 GO Terms


In [4]:
def make_ranked_df(grouped_loci):
    df = build_loci.get_sig_df(grouped_loci)
    
    # set insiginifcant loci to the center rank of the gene list
    df['logfc'] = df.where((abs(df['logfc']) <= 2) | (df['p_value'] <= 0.05))['logfc'].replace(np.nan, 0)
    
    # Right now FC ratios are light/heavy
    # light == Tcell and heavy == RAG
    # GSEA goes from low to High, so negative logFC values will come first
    # This will give genes high in RAG highest priority
    # Switching the sign with put the genes highest in Tcell mice up first
    df['logfc'] = -1 * df['logfc']
    
    # filter out the mouse and human genes
    mh_loci = build_loci.get_annotation_df(grouped_loci).query('mouse_human').index.tolist()
    df = df.query('~(id in {})'.format(mh_loci))

    
    # prep the ranking dataframe to gesapy specs.
    rank_df = df[['id', 'logfc']].rename(columns={'id': 'gene_name', 'logfc': 'rank'})
    
    return rank_df.sort_values('rank').reset_index(drop=True)


In [5]:
def make_go2Gene_map(grouped_loci, ontology='MF'):
    ontology_map = {'MF': 'molecular_function', 'BP': 'biological_process', 'CC': 'cellular_component'}
    
    out = defaultdict(set)    
    for l in grouped_loci:
        if 'go' in l.annotations and l.avg_ratio != 0:
            for go in l.annotations['go']:
                if obodag[go].namespace == ontology_map[ontology]:
                    out[go].add(l.cluster_id)
                    for parent in obodag[go].get_all_parents():
                        if obodag[parent].namespace == ontology_map[ontology]:
                            out[parent].add(l.cluster_id)
                
    return out

In [6]:
def filter_go2gene_map(go_locus):
    
    # Remove "very broad" gene sets. Arbitrary definition: gene sets that emcompass >50% of all IDs
    all_ids = set(chain(*go_locus.values()))
    go_locus = {key: value for (key, value) in go_locus.items() if len(value) / len(all_ids) <= 0.5}

    # Remove terms with less than 5 members: changed from 10 to 5 becasue small #s of proteins compared
    # to what you would find wiht genes
    go_locus = {key: value for (key, value) in go_locus.items() if len(value) >= 5}

    # Remove child terms with identical gene sets as their parents
    to_remove = set()
    for parent in go_locus.keys():
        # If child term has exact same members as parent, remove
        child_ids = [x.id for x in obodag[parent].children if x.id in go_locus.keys()]
        for child in child_ids:
            if go_locus[child] == go_locus[parent]:
                to_remove.add(child)
    go_locus = {key: value for (key, value) in go_locus.items() if key not in to_remove}

    # Remove sibling terms with identical gene sets
    to_remove = set()
    for brother in go_locus.keys():
        for parent in obodag[brother].parents:
            siblings = set([y.id for y in parent.children])
            siblings.remove(brother)
            for sibling in siblings:
                if sibling in go_locus.keys() and go_locus[brother] == go_locus[sibling]:
                    to_remove.add(sibling)
    go_locus = {key: value for (key, value) in go_locus.items() if key not in to_remove}

    return go_locus

In [7]:
def gomap_to_csv(go2gene, out_file = 'test.tsv'):

    out = ""
    for term, loci in go2gene.items():
        out += "{}\t".format(term)
        out += "{}\t".format(obodag[term].name)
        out += '\t'.join(list(map(str,loci)))
        out += '\n'
        
    with open(out_file, 'w') as fout:
        fout.write(out)

In [8]:
def run_go_gsea(rank_df, g2g_map, seed):
    """
    A ranked df and go2gene mapping returns the result dataframe for GSEA against all go-Terms
    
    loci must be grouped such that avg_ratio and p-values are correct for 1 phenotype
    see rt_unenr_grouped_loci above for example
    """
    import gseapy as gp
        
    # save the go 2 gene map, since gseapy doesn't seem to be able to use one already in memory
    gomap_to_csv(g2g_map, 'temp.gmt')
    
    res = gp.prerank(rnk=rank_df, gene_sets='temp.gmt', outdir='tmp', min_size = 5, max_size=500, 
                     graph_num = len(g2g_map), savefigs=False, seed=seed)
      
    def get_go_name(term):
        return obodag[term].name
    
    res['name'] = res.index.map(get_go_name)
    
    return res.sort_values('nes', ascending=False)

In [9]:
def plot_gsea_result(row, rank):
    return gp.gsea_plot.gsea_plot(rank, row['name'], row.hit_index, row.nes, row.pval, row.fdr, row.rank_ES, phenoPos='Tcell', phenoNeg='RAG')

In [10]:
mf_map = make_go2Gene_map(unenr_grouped_loci)
mf_map_f = filter_go2gene_map(mf_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(mf_map), len(mf_map_f)))

Unfiltered: 516	Filtered: 196


In [11]:
bp_map = make_go2Gene_map(unenr_grouped_loci, 'BP')
bp_map_f = filter_go2gene_map(bp_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(bp_map), len(bp_map_f)))

Unfiltered: 612	Filtered: 196


In [ ]:
unen_rank =  make_ranked_df(unenr_grouped_loci)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

mf_un_res = run_go_gsea(unen_rank, mf_map_f, seed=1111)
bp_un_res = run_go_gsea(unen_rank, bp_map_f, seed=2222)

117 gene_sets have been filtered out when max_size=500 and min_size=5
79 gene_sets used for further calculating
Start to compute enrichment socres...................... Thu Sep 22 19:08:29 2016
Start to compute esnulls................................ Thu Sep 22 19:08:29 2016
......This step might take a while to run. Be patient...
Start to compute pvals.................................. Thu Sep 22 19:08:42 2016
start to compute fdrs................................... Thu Sep 22 19:08:42 2016
Statistial testing finished............................. Thu Sep 22 19:08:42 2016
Start to generate gseapy reports, and produce figures... Thu Sep 22 19:08:42 2016
Congratulations. GSEAPY run successfully................
The Job is done.................................Goodbye! Thu Sep 22 19:08:42 2016
95 gene_sets have been filtered out when max_size=500 and min_size=5
101 gene_sets used for further calculating
Start to compute enrichment socres...................... Thu Sep 22 19:08:43 2016
Start 

In [ ]:
mf_un_TC = mf_un_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_un_RAG = mf_un_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

bp_un_TC = bp_un_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_un_RAG = bp_un_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

### Molecular Function - Up in Tcell

In [ ]:
res_cols = ['name', 'es', 'nes', 'pval', 'fdr', 'gene_set_size', 'matched_size']

In [ ]:
mf_un_TC[res_cols]

In [ ]:
sns.set_style('dark')
plot_gsea_result(mf_un_res.iloc[0], unen_rank)

In [ ]:
sns.set_style('dark')
plot_gsea_result(mf_un_res.iloc[2], unen_rank)

### Biological Process - Up in Tcell

In [ ]:
bp_un_TC[res_cols]

In [ ]:
plot_gsea_result(bp_un_res.loc['GO:0016052'], unen_rank)

### Molecular Function - Up in RAG

In [ ]:
mf_un_RAG[res_cols]

In [ ]:
plot_gsea_result(mf_un_res.loc['GO:0016817'], unen_rank)

### Biological Process - Up in RAG

In [ ]:
bp_un_RAG[res_cols]

In [ ]:
plot_gsea_result(bp_un_res.iloc[0], unen_rank)

## GSEA on BioGlyCMK Enriched Samples

In [ ]:
enr_rank =  make_ranked_df(enr_grouped_loci)

In [ ]:
# [l for l in enr_grouped_loci if l.cluster_id in enr_rank['gene_name'].tolist()]
enr_mf_map = make_go2Gene_map(enr_grouped_loci)
enr_mf_map_f = filter_go2gene_map(enr_mf_map)
print('MF Unfiltered: {}\tFiltered: {}'.format(len(enr_mf_map), len(enr_mf_map_f)))

enr_bp_map = make_go2Gene_map(enr_grouped_loci, 'BP')
enr_bp_map_f = filter_go2gene_map(enr_bp_map)
print('BP Unfiltered: {}\tFiltered: {}'.format(len(enr_bp_map), len(enr_bp_map_f)))

In [ ]:
mf_enr_res = run_go_gsea(enr_rank, enr_mf_map_f, seed=1234)
bp_enr_res = run_go_gsea(enr_rank, enr_bp_map_f, seed=4321)

In [ ]:
mf_enr_TC = mf_enr_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_enr_RAG = mf_enr_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

bp_enr_TC = bp_enr_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_enr_RAG = bp_enr_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

### Up in T-Cell: Molecular Function

In [ ]:
mf_enr_TC[res_cols]

In [ ]:
obodag.query_term('GO:0016810')

In [ ]:
plot_gsea_result(mf_enr_res.loc['GO:0004175'], enr_rank)

In [ ]:
plot_gsea_result(mf_enr_res.loc['GO:0016787'], enr_rank)

### Up in T-Cell: Biological Process

In [ ]:
bp_enr_TC[res_cols]

In [ ]:
plot_gsea_result(bp_enr_res.loc['GO:0006508'], enr_rank)

In [ ]:
plot_gsea_result(bp_enr_res.loc['GO:0019538'], enr_rank)

### Up in RAG: Molecular Function

In [ ]:
mf_enr_RAG[res_cols]

In [ ]:
plot_gsea_result(mf_enr_res.loc['GO:0016781'], enr_rank)

### Up in RAG: Biological Process

In [ ]:
bp_enr_RAG[res_cols]

In [ ]:
plot_gsea_result(bp_enr_res.loc['GO:0006457'], enr_rank)